In [ ]:
#| default_exp argument_validators

In [ ]:
#| export
from typing import Protocol, Any, runtime_checkable
from pymoq.core import AnyCallable

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import test_fail

# Validators

> Collection of methods to validate specific call arguments.

Goal: Evaluate whether a call like `f(1,"s")` matches any signature-pattern. A signature pattern might be defined like `f(1, str)`. This should match any call that passes the exact value one for the first argument and any object of type str in the second.

## Argument validators

We break the task down to validating a single argument. The signature of such an ArgumentValidator should look like :

In [ ]:
#| export
@runtime_checkable
class ArgumentValidator(Protocol):
    @property
    def name() -> str|None:
        ...
        
    "Interface for all argument validators."
    def is_valid(self, argument: Any) -> bool:
        ...

The most flexibility can be achieved by constructing an ArgumentValidator that evaluates an arbitrary function:

In [ ]:
#| export
class ArgumentFunctionValidator:
    "Validate an argument by evaluating an arbitrary function"
    def __init__(self, func: AnyCallable[bool], name: str|None = None):
        self._func = func
        self._name = name
        
    @property
    def name(self) -> str|None:
        return self._name
        
    def is_valid(self, argument: Any) -> bool:
        return self._func(argument)
    
assert isinstance(ArgumentFunctionValidator, ArgumentValidator), "ArgumentFunctionValidator does not implement the ArgumentValidator-Protocol"

This could now be used like:

In [ ]:
any_int = ArgumentFunctionValidator(lambda v: isinstance(v, int))

In [ ]:
assert any_int.is_valid(1)
assert not any_int.is_valid(1.1)
assert not any_int.is_valid("string")

In later stages there should be convenience methods around creating such argument validators. E.g. `from_type(some_type)` for making the above easier.

## Signature validators

A signature validator is simply a collection of argument validators. Its `is_valid` methods checks for a given list of arguments if all argument validators return valid.

In [ ]:
#| export
class SignatureValidator:
    def __init__(self, argument_validators: list[ArgumentValidator]):
        self.argument_validators = argument_validators
        self._named_validators = {validator.name: validator
                                  for validator in self.argument_validators
                                  if validator.name is not None }
        if len(self._named_validators) != len(set(self._named_validators.keys())):
            raise ValueError(f"List of argument validators contains duplicate names: {list(self._named_validators.keys())}")
        
    def is_valid(self, *args: list[Any], **kwargs: dict[str, Any]) -> bool:
        if len(args) > len(self.argument_validators): return False
    
        # positional arguments
        if not all(argument_validator.is_valid(arg) for arg,argument_validator in zip(args, self.argument_validators)):
            return False
        
        # named arguments
        for name,value in kwargs.items():
            if name not in self._named_validators: return False
        
            if not self._named_validators[name].is_valid(value): return False
        
        return True

In [ ]:
s = SignatureValidator([any_int, any_int])

Calling a signature validator with only positional arguments works as expected:

In [ ]:
assert s.is_valid(1,1)
assert not s.is_valid(1,"1")
assert not s.is_valid("1", 1)

Named arguments:

In [ ]:
named_arg_validator = ArgumentFunctionValidator(lambda v: isinstance(v, int), name="named")
s = SignatureValidator([any_int, named_arg_validator])

In [ ]:
assert s.is_valid(1, named=1)
assert not s.is_valid(1, named="1")

#### Edge cases

Argument validators contain more than one element with the same name

In [ ]:
# Why does this not throw an error??
SignatureValidator([named_arg_validator, named_arg_validator])

<__main__.SignatureValidator>

More arguments than validators:

In [ ]:
assert not s.is_valid(1,1,1)

# Build library

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()